<a href="https://colab.research.google.com/github/meghutch/Breast-Cancer-Classification-Wisconsin/blob/master/Breast_Cancer_Classification_Clean_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Breast Cancer Classification**

**Author:** Meg Hutch

**Date:** October 22, 2019


**Objective:** Classify Breast Cancer Tumnors as Malignant or Benign from the Breast Cancer Wisconin Dataset downloaded fromn https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

**Additional reference:** https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

**Dataset:** The following is the given data descriptions: 

**Attribute Information:**

Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image.

1.   ID Number
2.   Diagnosis (M = malignant, B = benign)

Ten real-valued features are computed for each cell nucleus (3-32):

3.  radius (mean of distances from center to points on the perimeter)
4.  texture (standard deviation of gray-scale values)
5.  perimeter
6.  area
7.  smoothness (local variation in radius lengths)
8.  compactness (perimeter^2 / area - 1.0)
9.  concavity (severity of concave portions of the contour)
10. concave points (number of concave portions of the contour)
11. symmetry
12. fractal dimension ("coastline approximation" - 1)

The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.

**WIP Updates**

**11.01.2019: MH implemented the code for logisitic regression, will need to add random forest, and also ensure the neural network is okay**

**11.04.2019: MH is not sure if I need the train_x or val_x to also be converted to long format ; I'm having problems getting the models to run due to dimension problems**

**11.05.2019: MH will try revising the code for xb, yb and in regards to the data loader -- I think this is the problem. First though, I will save what I've done to github**

**Update: Can't figure out how to upload to github, but I created a cleaner version of this code. I'm thinking about removing the validaiton set, since I think this reduces the size, but I'm a bit weary of doing so.**

**Need to also implement random forest classifier**

**11.07.2019: I implemented random forest classifier and am created a Clean_2 where I'm going to remove the seperate splitting function from the neural networks so that I am ensuring that I'm testing all on the same set. I am also going to examine the distributions of the split -- this may be imbalanced due to the high Random Forest Accuracy**

**Need to analyze evaluation metrics for the neural network. Check this site: https://github.com/betogaona7/dermatologist-AI/blob/master/dermatologist_AI.ipynb**

**11.08.2019: It wouldn't hurt to re-evaluate everything; especially since AUC for neural network is pretty low, but I think I have managed to implement all of the pieces**

In [0]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

# Connect Colab to google drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Import Data
tumor = pd.read_csv('/content/drive/My Drive/Projects/Breast_Cancer_Wisconsin/data.csv')

# View data
tumor.head(10)

# **Explore Data**

First, we will examine the worst values obtained from each patient.

In [0]:
tumor.diagnosis.value_counts().plot(kind="bar")
count_dx = tumor.groupby(['diagnosis']).size()
print('Total Number of Patients:', len(tumor.index))
print('Number Diagnosed:', count_dx)
print('Percent Benign: {:.1f}'.format(357/len(tumor.index)))
print('Percent Malignant: {:.1f}'.format(212/len(tumor.index)))


In [0]:
# Create a new dataframe to just contain columns of interset
tumor_plots = tumor[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']]

# Generically define how many plots along and across
ncols = 3
nrows = int(np.ceil(len(tumor_plots.columns) / (1.0*ncols)))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 10))

# Lazy counter so we can remove unwanted axes
counter = 0
for i in range(nrows):
    for j in range(ncols):

        ax = axes[i][j]

        # Plot when we have data
        if counter < len(tumor_plots.columns):

            ax.hist(tumor_plots[tumor_plots.columns[counter]], bins=50, color='blue', alpha=0.5, label='{}'.format(tumor_plots.columns[counter]))
            ax.set_xlabel('x')
            ax.set_ylabel('PDF')
            leg = ax.legend(loc='upper left')
            leg.draw_frame(False)

        # Remove axis when we no longer have data
        else:
            ax.set_axis_off()

        counter += 1

plt.show()

# **Correlations for Feature Selection**

I'll eventually have to learn how to look into this more, as of now, I'm just going to include all features

In [0]:
# Basic correlogram
#sns.pairplot(tumor_plots)
#plt.show()

In [0]:
tumor.head()

# **Logistic Regression**

We will first try and assess classification using a simple logistic regression - this will also serve as a bench mark once we develop our neural network classifier

These steps were followed from the following tutorial: https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python

In [0]:
# Packages 
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Create dummy variables for diagnoses

In [0]:
tumor['diagnosis'] = tumor.diagnosis.map({'B':0, 'M':1})

Create data frames into features and labels


In [0]:
# create x to represent the input features; y is the label; 
x = tumor[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']]
y = tumor.diagnosis # This is output of our training data

Split the data into testing and training

In [0]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

The way I named the labels above may be a bit confusing. 

But the way the splits above work, train = training data, test = testing data

In [0]:
# Assess the trainig and testing sets previously created
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Examine the Class Distributions - We can see that in both the testing and the training set, 37% of the cases are malignant. I feel like this a good split because within the whole dataset there is 40% malignant cases. 

In [0]:
# Malignant = 1; Count the number of malignant and determine the percentage (traning has 426 values, testing has 143)
print('Malginant Cases in Training Set:', np.count_nonzero(y_train == 1))
print('Malginant Cases in Testing Set:', np.count_nonzero(y_test == 1))

# Percents
print('% Malginant Cases in Training Set:', round(np.count_nonzero(y_train == 1)/426*100,2))
print('% Malginant Cases in Testing Set:', round(np.count_nonzero(y_test == 1)/143*100,2))

Develop the logistic regression model

In [0]:
# instantiate the model (using default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

In [0]:
y_pred

**Model Evaulation using Confusion Matrix**

In [0]:
# import the metrics class
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

The confusion matrix generated abouve is in the form of an array. Diagonal values represent accurate predictions, while non-diagnonal elements are inaccurate predictions. The diagnoal starting with the top left to the bottom right hand corner are the actual predictions, while the bottom left corner to the top right corner are incorrect predictions.

In [0]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

**ROC**

The Reciever Operating Characteristic (ROC) curve is a plot of the true positive rate against the false positive rate. It shows the tradeoff between sensitivity and specificty

In [0]:
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()
print(round(auc,3))

# **Random Forest Classification**

I am following the tutorial here: https://towardsdatascience.com/random-forest-in-python-24d0893d51c0 and https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/

We already split the data into training and testing above. We will use this same split for the random forest model.

**Train the Model**

In [0]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train)

**Make Predictions on the Test Set**

In [0]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Probabilities for each class
rf_probs = rf.predict_proba(X_test)[:, 1]

# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Results indicate that our estimate is off by 0.06





We make class predictions (predict) as well as predicted probabilities (predict_proba) to calculate the ROC AUC. Once we have the testing predictions, we can calculate the ROC AUC. https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76

In [0]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(y_test, rf_probs)

roc_value

**Evaluating the Algorithm**

For classification problems the metrics used to evaluate an algorithm are accuracy, confusion matrix, precision recall, and F1 values. Execute the following script to find these values:

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [0]:
# Cross Fold Validation
rfc_cv_score = cross_val_score(rf, x, y, cv=10, scoring='roc_auc') #https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652 

Below, we can examine the confusion matrix, the classification report contraining precision, recall, f1-score, and supoprt, All AUC scores, and the Mean AUC Score.

The **confusion matrix** is useful for giving false positives and false negatives. 

The **ROC Curve** plots out the true positive rate vs the false positive rate at various thresholds

The **Roc AUC Scoring** used in the cross-validaiton model shows the area under the ROC curve 


In [0]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, predictions))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, predictions))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())
print('\n')
print("=== Accuracy Score ===")
print("Accuracy:", accuracy_score(y_test, predictions))

In [0]:
# Probabilities for each class
# Note from scikit learn re: fpr and tpr: "Since the thresholds are sorted from low to high values, they are reversed upon returning them to ensure they correspond to both fpr and tpr, which are sorted in reversed order during their calculation."
rf_probs = rf.predict_proba(X_test)[:, 1]
fpr, tpr, _ = metrics.roc_curve(y_test,  rf_probs)
auc = metrics.roc_auc_score(y_test, rf_probs)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

The Random forest classifier has a slightly high accuracy than logistic regression 92% vs 94%. Additionally, for the Random Forest Classifier the AUC was 0.984 after 10 fold cross validation, though the first run had an AUC of .99. For the Logistic Regression Classifier, was 0.984, the same as after the 10 fold cross validation. 

## **PyTorch Neural Network for Classification**

We will evaluate the use of a neural network classifier on the testing data as developed above and compare the outputs to the logistic regression and random forest classifiers above


In [0]:
# Import PyTorch packages
import torch
from torch import nn
from torchvision import datasets, transforms
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import TensorDataset
import torch.nn.functional as F

**Format the Training Dataset**

In [0]:
# Convert data into arrays
xb = np.array(X_train, dtype = "float32")
yb = np.array(y_train, dtype= "float32")

# Convert arrays into tensors
xb = torch.from_numpy(xb)
yb = torch.from_numpy(yb)

#Combine the arrays 
trainloader = TensorDataset(xb, yb) 

# Define the batchsize
batch_size=25

# Training Loader
trainloader = DataLoader(trainloader, 
                         batch_size)

**Format the Testing Dataset**

In [0]:
# Convert data into arrays
xb = np.array(X_test, dtype = "float32")
yb = np.array(y_test, dtype= "float32")

# Convert arrays into tensors
xb = torch.from_numpy(xb)
yb = torch.from_numpy(yb)

#Combine the arrays 
test = TensorDataset(xb, yb) 

# Define the batchsize
batch_size=25

# Training Loader
testloader = DataLoader(test, 
                         batch_size)

**Create Neural Network Model**

In [0]:
# Define the model with one hidden layer
model = nn.Sequential(nn.Linear(10, 5),
                      nn.ReLU(),
                      nn.Linear(5, 2),
                      nn.LogSoftmax(dim=1))

# Set optimizer and learning rate
#optimizer = optim.SGD(model.parameters(), lr=0.001)

# Could also use Adam optimizer; similar to stochastic gradient descent, but uses momentum which can speed up the actual fitting process, and it also adjusts the learning rate for each of the individual parameters in the model
optimizer = optim.Adam(model.parameters(), lr=0.003)

# Define the loss
criterion = nn.NLLLoss()

# Set 100 epochs to start
epochs = 100
for e in range(epochs):
    running_loss = 0
    for xb, yb in trainloader:

        # Flatten yb
        #yb = yb.view(yb.shape[0], -1)
        
        # Clear the gradients, do this because gradients are accumulated
        optimizer.zero_grad()
        
        # Training pass
        output = model.forward(xb)
        loss = criterion(output, yb.long()) # Loss calculated from the output compared to the labels 
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() # loss.item() gets the scalar value held in the loss. Running_loss = 0, 
        # += notation, says "Add a value and the variable and assigns the result to that variable." So, adds the running_loss (0) with loss.item and assigns to running_loss
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

The goal of validation is to measure the model's performance on data that isn't part of the training set. Performance here is up to the developer to define though. Typically, this is just accuracy, the percentage of classes the network predicted correctly. 

First, do a forward pass with one batch from the test set 

In [0]:
xb, yb = next(iter(testloader))

# Get the class probabilities 
ps = torch.exp(model(xb))

# Make sure the shape is appropriate
print(ps.shape)

With the probabilities, we can get the most likely class using the ps.topk method. This returns the k highest values. Since we just want the most likely class, we can use ps.topk(1). This returns a tuple of the top-k values and the top-k indices. If the highest value is the first element, we'll get back 4 as the index. 

In [0]:
top_p, top_class = ps.topk(1, dim=1)
# Look at the most likely classes for the first 20 examples
print(top_class[:20,:])

In [0]:
top_p

In [0]:
ps[:20] #we can see how to correpsonding probabilities were classified above

Now we can check if the predicted classes match the labels. This is simple to do by equating top_class and labels, but we have to be careful of the shapes. To get the equality to work out the way we want, top_class and the labels (yb) must have the same shape.

In [0]:
equals = top_class == yb.view(*top_class.shape) 
equals

In [0]:
top_class = top_class.view(*yb.shape) 
top_class

In [0]:
# Cast top_class to float32
top_class = top_class.to(dtype=torch.float32)

Now we need to calculate the correct predictions. 

equals has binary values, either 0 or 1. This means that if we just sum up all the values and divide by the total number of values, we get the percentage of correct predictions. This is the same operation as taking the mean, so we can get the accuracy with a call to torch.mean. 

So we'll need to convert equals to a float tensor. Note that when we take torch.mean it returns a scalar tensor, to get the actual value as a float we'll need to do accuracy.item()

In [0]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

**Evaulating the Model**

In [0]:
from fastai.vision import *
roc_auc = auc_roc_score(yb, top_class)
roc_auc

In [0]:
top_class.shape

**Plot ROC**

In [0]:
from sklearn.metrics import roc_curve, auc
# Compute ROC curve
fpr, tpr, thresholds = roc_curve(yb, top_p, pos_label=1)

# Compute ROC area
roc_auc = auc(fpr, tpr)
print('ROC area is {0}'.format(roc_auc))
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

In [0]:
# Calculating True Positive Rate (Sensitivity) and False Positive Rate (Sensitivity)
# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal

CM = confusion_matrix(yb, top_class)

TN = CM[0][0] # True Negative is high - is this because our model has greater sensitivty
FN = CM[1][0] # False Negative
TP = CM[1][1] # True Positive
FP = CM[0][1] # False POsitive 

In [0]:
FN